### 🔹 Modeling

In [165]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

pd.set_option('display.max_columns', None)

In [166]:
train = pd.read_csv('Datasets/train.csv')
features = pd.read_csv('Datasets/features.csv')
stores = pd.read_csv('Datasets/stores.csv')
test = pd.read_csv('Datasets/test.csv')

In [167]:
feature_store = features.merge(stores, how='left', on = "Store")

feature_store['Date'] = pd.to_datetime(feature_store['Date'])
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

feature_store['Day'] = feature_store['Date'].dt.day
feature_store['Week'] = feature_store['Date'].dt.isocalendar().week
feature_store['Month'] = feature_store['Date'].dt.month
feature_store['Year'] = feature_store['Date'].dt.year

In [168]:
train_merged= pd.merge(train, feature_store, on=['Store', 'Date', 'IsHoliday'], how='left')
display(train_merged.head())

test_merged = pd.merge(test, feature_store, on=['Store', 'Date', 'IsHoliday'], how='left')
display(test_merged.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,5,5,2,2010
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315,12,6,2,2010
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315,19,7,2,2010
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315,26,8,2,2010
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315,5,9,3,2010


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315,2,44,11,2012
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315,9,45,11,2012
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315,16,46,11,2012
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315,23,47,11,2012
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315,30,48,11,2012


In [169]:
train_merged['Days_to_Thansksgiving'] = (pd.to_datetime(train_merged["Year"].astype(str)+"-11-24", format="%Y-%m-%d") - pd.to_datetime(train_merged["Date"], format="%Y-%m-%d")).dt.days.astype(int)
train_merged['Days_to_Christmas'] = (pd.to_datetime(train_merged["Year"].astype(str)+"-12-24", format="%Y-%m-%d") - pd.to_datetime(train_merged["Date"], format="%Y-%m-%d")).dt.days.astype(int)

In [170]:
# Define holiday date lists
superbowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08']
labor_dates = ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06']
thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29']
christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']

# Create binary indicator columns
train_merged['SuperBowlWeek'] = train_merged['Date'].isin(superbowl_dates).astype(int)
train_merged['LaborDay'] = train_merged['Date'].isin(labor_dates).astype(int)
train_merged['Thanksgiving'] = train_merged['Date'].isin(thanksgiving_dates).astype(int)
train_merged['Christmas'] = train_merged['Date'].isin(christmas_dates).astype(int)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\534379698.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train_merged['SuperBowlWeek'] = train_merged['Date'].isin(superbowl_dates).astype(int)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\534379698.py:9: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train_merged['LaborDay'] = train_merged['Date'].isin(labor_dates).astype(int)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\534379698.py:10: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future

In [171]:
train_merged.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year,Days_to_Thansksgiving,Days_to_Christmas,SuperBowlWeek,LaborDay,Thanksgiving,Christmas
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,5,5,2,2010,292,322,0,0,0,0
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315,12,6,2,2010,285,315,1,0,0,0
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315,19,7,2,2010,278,308,0,0,0,0
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315,26,8,2,2010,271,301,0,0,0,0
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315,5,9,3,2010,264,294,0,0,0,0


In [172]:
test_merged['Days_to_Thansksgiving'] = (pd.to_datetime(test_merged["Year"].astype(str)+"-11-24", format="%Y-%m-%d") - pd.to_datetime(test_merged["Date"], format="%Y-%m-%d")).dt.days.astype(int)
test_merged['Days_to_Christmas'] = (pd.to_datetime(test_merged["Year"].astype(str)+"-12-24", format="%Y-%m-%d") - pd.to_datetime(test_merged["Date"], format="%Y-%m-%d")).dt.days.astype(int)

In [173]:
# Define holiday date lists
superbowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08']
labor_dates = ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06']
thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29']
christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']

# Create binary indicator columns
test_merged['SuperBowlWeek'] = test_merged['Date'].isin(superbowl_dates).astype(int)
test_merged['LaborDay'] = test_merged['Date'].isin(labor_dates).astype(int)
test_merged['Thanksgiving'] = test_merged['Date'].isin(thanksgiving_dates).astype(int)
test_merged['Christmas'] = test_merged['Date'].isin(christmas_dates).astype(int)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\2392138159.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_merged['SuperBowlWeek'] = test_merged['Date'].isin(superbowl_dates).astype(int)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\2392138159.py:9: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_merged['LaborDay'] = test_merged['Date'].isin(labor_dates).astype(int)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\2392138159.py:10: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future 

In [174]:
test_merged.head()

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year,Days_to_Thansksgiving,Days_to_Christmas,SuperBowlWeek,LaborDay,Thanksgiving,Christmas
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315,2,44,11,2012,22,52,0,0,0,0
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315,9,45,11,2012,15,45,0,0,0,0
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315,16,46,11,2012,8,38,0,0,0,0
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315,23,47,11,2012,1,31,0,0,1,0
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315,30,48,11,2012,-6,24,0,0,0,0


In [175]:
train_merged.fillna(0, inplace = True)

test_merged['CPI'].fillna(test_merged['CPI'].mean(), inplace = True)
test_merged['Unemployment'].fillna(test_merged['Unemployment'].mean(), inplace = True)

test_merged.fillna(0, inplace = True)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\1726027120.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged['CPI'].fillna(test_merged['CPI'].mean(), inplace = True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_10284\1726027120.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

In [176]:
train_merged.isnull().sum()

Store                    0
Dept                     0
Date                     0
Weekly_Sales             0
IsHoliday                0
Temperature              0
Fuel_Price               0
MarkDown1                0
MarkDown2                0
MarkDown3                0
MarkDown4                0
MarkDown5                0
CPI                      0
Unemployment             0
Type                     0
Size                     0
Day                      0
Week                     0
Month                    0
Year                     0
Days_to_Thansksgiving    0
Days_to_Christmas        0
SuperBowlWeek            0
LaborDay                 0
Thanksgiving             0
Christmas                0
dtype: int64

In [177]:
test_merged.isnull().sum()

Store                    0
Dept                     0
Date                     0
IsHoliday                0
Temperature              0
Fuel_Price               0
MarkDown1                0
MarkDown2                0
MarkDown3                0
MarkDown4                0
MarkDown5                0
CPI                      0
Unemployment             0
Type                     0
Size                     0
Day                      0
Week                     0
Month                    0
Year                     0
Days_to_Thansksgiving    0
Days_to_Christmas        0
SuperBowlWeek            0
LaborDay                 0
Thanksgiving             0
Christmas                0
dtype: int64

In [178]:
train_merged.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Day', 'Week',
       'Month', 'Year', 'Days_to_Thansksgiving', 'Days_to_Christmas',
       'SuperBowlWeek', 'LaborDay', 'Thanksgiving', 'Christmas'],
      dtype='object')

In [179]:
X_baseline = train_merged[['Store','Dept','IsHoliday','Size','Week','Type','Year','Day']]
y = train_merged['Weekly_Sales']

In [180]:
X_baseline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Store      421570 non-null  int64 
 1   Dept       421570 non-null  int64 
 2   IsHoliday  421570 non-null  bool  
 3   Size       421570 non-null  int64 
 4   Week       421570 non-null  UInt32
 5   Type       421570 non-null  object
 6   Year       421570 non-null  int32 
 7   Day        421570 non-null  int32 
dtypes: UInt32(1), bool(1), int32(2), int64(3), object(1)
memory usage: 18.5+ MB


In [ ]:
X_baseline = X_baseline
X_baseline['IsHoliday'] = X_baseline['IsHoliday'].astype(int)

numeric_features = ['Store', 'Dept', 'Size', 'Week', 'Year', 'Day']
categorical_features = ['Type']
boolean_features = ['IsHoliday']

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('bool', 'passthrough', boolean_features)
])

models = {
    'LGBM': lgb.LGBMRegressor(random_state=0),
    'RandomF': ensemble.RandomForestRegressor(random_state=0)
}

param_grids = {
    'LGBM': {
        'model__n_estimators': [300, 600],
        'model__num_leaves': [31, 63],
        'model__learning_rate': [0.05, 0.1],
    },
    'RandomF': {
        'model__n_estimators': [300, 600],
        'model__max_depth': [None, 20],
        'model__max_features': ['sqrt', 0.5],
    }
}

cv = KFold(n_splits=5, shuffle=True, random_state=0)

results_summary = []

for name, model in models.items():
    print(f"Running Grid Search for {name}...")

    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grids[name],
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        verbose=2
    )

    grid.fit(X_baseline, y)

    results_summary.append({
        'model': name,
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    })

results_df = pd.DataFrame(results_summary).sort_values('best_score', ascending=False)
print(results_df)


Running Grid Search for LGBM...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


KeyboardInterrupt: 

In [213]:
# X = train_merged.copy().drop(columns=['Date', 'Weekly_Sales'])
# features = X.columns

# numeric_features = [feature for feature in features if X[feature].dtype != 'object' and X[feature].dtype != 'bool']
# categorical_features = [feature for feature in features if X[feature].dtype == 'object']
# boolean_features = [feature for feature in features if X[feature].dtype == 'bool']

features = X_baseline.columns

numeric_features = [feature for feature in features if X_baseline[feature].dtype != 'object' and X_baseline[feature].dtype != 'bool']
categorical_features = [feature for feature in features if X_baseline[feature].dtype == 'object']
boolean_features = [feature for feature in features if X_baseline[feature].dtype == 'bool']


print(numeric_features)
print(categorical_features)
print(boolean_features)

def bool_to_int(x):
    return x.astype(int)

# numeric_transformer = Pipeline([('scaler', StandardScaler())])
numeric_transformer = Pipeline([('scaler', 'passthrough')])
categorical_transformer = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
boolean_transformer = Pipeline([('to_int', FunctionTransformer(bool_to_int))])
    
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('bool', boolean_transformer, boolean_features)
])

cv = KFold(n_splits=5, shuffle=True, random_state=0)

['Store', 'Dept', 'Size', 'Week', 'Year', 'Day']
['Type']
['IsHoliday']


In [214]:
model = lgb.LGBMRegressor(random_state=0)

param_grid = {
    'model__n_estimators': [300, 600],
    'model__num_leaves': [31, 63],
    'model__learning_rate': [0.05, 0.1]
}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

print("Running Grid Search for LightGBM...")
grid.fit(X_baseline, y)
print("Best RMSE:", -grid.best_score_)
print("Best Params:", grid.best_params_)

joblib.dump(grid.best_estimator_, "models/best_lgbm_pipeline.joblib")


Running Grid Search for LightGBM...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 266
[LightGBM] [Info] Number of data points in the train set: 421570, number of used features: 10
[LightGBM] [Info] Start training from score 15981.258121
Best RMSE: 3475.994849005524
Best Params: {'model__learning_rate': 0.1, 'model__n_estimators': 600, 'model__num_leaves': 63}


['models/best_lgbm_pipeline.joblib']

In [215]:
model = xgb.XGBRegressor(random_state=0, objective='reg:squarederror')

param_grid = {
    'model__n_estimators': [300, 600],
    'model__max_depth': [4, 8],
    'model__learning_rate': [0.05, 0.1]
}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=2
)

print("Running Grid Search for XGBoost...")
grid.fit(X_baseline, y)
print("Best RMSE:", -grid.best_score_)
print("Best Params:", grid.best_params_)

joblib.dump(grid.best_estimator_, "models/best_xgbm_pipeline.joblib")


Running Grid Search for XGBoost...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best RMSE: 2944.2376849739157
Best Params: {'model__learning_rate': 0.1, 'model__max_depth': 8, 'model__n_estimators': 600}


['models/best_xgbm_pipeline.joblib']

In [216]:
model = cb.CatBoostRegressor(random_state=0, verbose=False)

param_grid = {
    'model__iterations': [300, 600],
    'model__depth': [4, 8],
    'model__learning_rate': [0.05, 0.1]
}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=2
)

print("Running Grid Search for CatBoost...")
grid.fit(X_baseline, y)
print("Best RMSE:", -grid.best_score_)
print("Best Params:", grid.best_params_)

joblib.dump(grid.best_estimator_, "models/best_catbm_pipeline.joblib")

Running Grid Search for CatBoost...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best RMSE: 5322.27387479951
Best Params: {'model__depth': 8, 'model__iterations': 600, 'model__learning_rate': 0.1}


['models/best_catbm_pipeline.joblib']

In [250]:
X_prep = preprocessor.fit_transform(X_baseline)
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.3, random_state=42)

rfm = ensemble.RandomForestRegressor(n_estimators=60, max_depth=25, min_samples_split=3, min_samples_leaf=1)

rfm.fit(X_train, y_train)
rfm_pred = rfm.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, rfm_pred))

print(f"RFM RMSE: {rmse}")

rfm.fit(X_prep, y)

joblib.dump(rfm, "models/best_rfm_pipeline.joblib")

RFM RMSE: 3487.274389291161


['models/best_rfm_pipeline.joblib']

In [251]:
X_prep = preprocessor.fit_transform(X_baseline)
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.3, random_state=42)

etrm = ensemble.ExtraTreesRegressor(n_estimators=50, bootstrap = True, random_state = 0)

etrm.fit(X_train, y_train)
etrm_pred = etrm.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, etrm_pred))

print(f"ERM RMSE: {rmse}")

etrm.fit(X_prep, y)

joblib.dump(etrm, "models/best_etrm_pipeline.joblib")

ERM RMSE: 3762.2554766507556


['models/best_etrm_pipeline.joblib']

In [219]:
avg_pred = (etrm_pred + rfm_pred) / 2
print(np.sqrt(mean_squared_error(y_test, avg_pred)))

3475.769982484429


In [235]:
lgbm = joblib.load("models/best_lgbm_pipeline.joblib")

In [236]:
xgbm = joblib.load("models/best_xgbm_pipeline.joblib")

In [237]:
catbm = joblib.load("models/best_catbm_pipeline.joblib")

In [252]:
rfm = joblib.load("models/best_rfm_pipeline.joblib")

In [253]:
etrm = joblib.load("models/best_etrm_pipeline.joblib")

In [240]:
submission = pd.DataFrame()
submission['Id'] = test_merged.apply(lambda x : f"{x['Store']}_{x['Dept']}_{x['Date'].strftime('%Y-%m-%d')}", axis=1)
submission.head()

,Id
0,1_1_2012-11-02
1,1_1_2012-11-09
2,1_1_2012-11-16
3,1_1_2012-11-23
4,1_1_2012-11-30


In [257]:
test_prep = test_merged.drop(columns=['Date'])
y_pred_lgbm = lgbm.predict(test_prep)
y_pred_xgbm = xgbm.predict(test_prep)
y_pred_catbm = catbm.predict(test_prep)

d:\Projects\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [258]:
test_prep = preprocessor.transform(test_prep)
y_pred_rfm = rfm.predict(test_prep)

In [259]:
y_pred_etrm = etrm.predict(test_prep)

In [260]:
submission['Weekly_Sales'] = y_pred_lgbm
submission.to_csv("./submissions/lgbm_submission.csv", index=False)

In [261]:
submission['Weekly_Sales'] = y_pred_xgbm
submission.to_csv("./submissions/xgbm_submission.csv", index=False)

In [262]:
submission['Weekly_Sales'] = y_pred_catbm
submission.to_csv("./submissions/catbm_submission.csv", index=False)

In [263]:
submission['Weekly_Sales'] = y_pred_rfm
submission.to_csv("./submissions/rfm_submission.csv", index=False)

In [264]:
submission['Weekly_Sales'] = y_pred_etrm
submission.to_csv("./submissions/etrm_submission.csv", index=False)

In [265]:
submission['Weekly_Sales'] = (y_pred_etrm + y_pred_rfm) / 2
submission.to_csv("./submissions/avg_pred_submission.csv", index=False)